# Pre-select Filter
Three topologies will be discussed:
1. Parallel LC Bandpass Filter : very low inductance, hard to control.
2. Top-C Coupled resonator band pass filter
3. Lumped LC-filter : Convential series first bandpass filter

## Parallel LC Bandpass Filter

In [1]:
import math
f_0 = 98e6 # 98 MHz, center frequency of the FM broadcast band
B = 20e6 # 20 MHz, bandwidth (88-108 MHz)
Rin = 50 # 50 Ohm, input impedance (antenna impedance)
Rout = 50 # 50 Ohm, output impedance (input impedance of the amplifier)

R_L = Rin * Rout / (Rin + Rout) # l
Q = f_0 / B # Q factor
print(f"Q = {Q}")
X_0 = R_L / Q
ω = 2 * math.pi * f_0

L = X_0 / ω # inductance
print(f"Calculated L = {L} H")

# Select the actual part for the inductor:
L_act = 10e-9 # 10 nH, WE 744917110
Q_Lact = 39 # Q factor of the actual inductor
R_Lact = ω * L_act / Q_Lact # resistance of the actual inductor
print(f"R_Lact = {R_Lact} Ohm")

# Calculate the capacitance
C = 1 / (L_act * ω**2) # capacitance 
print(f"C = {C} F")

Q = 4.9
Calculated L = 8.28586750790792e-09 H
R_Lact = 0.15788516925733317 Ohm
C = 2.6374735433761393e-10 F


The 264pF capacitor for the pre-select filter will be constituated of a 47pF and a 220pF capacitor in parallel.  A 180pF+82pF is possible too, but these values are not available at Wurth Elektronik.

* L = 10nH, WE 744917110
* C1 = 47pF, WE 885012006055
* C2 = 220pF, WE 885012006059

The inductor value is very low.  Every piece of PCB-trace added will have a significant impact.

## Top-C Coupled resonator band pass filter
This topology is proposed by the lecturer in the video series.

[Filter Design Part 11. How to Design Bandpass Filter BPF with Top-Coupled Capacitive Resonator.](https://www.youtube.com/watch?v=djknXHFLqsA) gives some details about when to use this topology.  
* low side roll/off 18dB/octave
* high side roll/off 6dB/octave.  

This makes this filter more suitable for low-side injection in which the frequency of the local oscillator is lower than the frequency of the signal.  The unwanted image frequency is lower than the wanted frequency, so it gets more suppression.

In [2]:
import math
# Calculations from [Filter Design Part 11. How to Design Bandpass Filter BPF with Top-Coupled Capacitive Resonator.](https://www.youtube.com/watch?v=djknXHFLqsA)
# The same calculations can also be found on [RF Circuit Design, Second Edition -- Christopher Bowick, Chapter 2, Coupling of resonant circuits]

f_0 = 98e6 # 98 MHz, center frequency of the FM broadcast band
B = 20e6 # 20 MHz, bandwidth (88-108 MHz)
Rs = Rl = 50 # 50 Ohm, source and load impedance

Q_t = f_0 / B # Q factor of the total filter
Q_r = Q_t * math.sqrt(2) # Q factor of a single resonator
X_L1 = Rs / Q_r # inductive reactance
X_C1 = X_L1 # capacitive reactance
L_1 = X_L1 / (2 * math.pi * f_0) # inductance of the resonator
print(f"L_1 = {L_1} H")
C_1 = 1 / (2 * math.pi * f_0 * X_C1) # capacitance of the resonator
print(f"C_1 = {C_1} F")
C_c = C_1 / Q_r # coupling capacitance
print(f"C_c = {C_c} F")

L_1 = 1.1717986205709938e-08 H
C_1 = 2.2507907903927656e-10 F
C_c = 3.248060063099905e-11 F


When simulating this filter in LTSpice, the resulting frequency response shows a center frequency considerably lower than 98MHz.  I have no idea why.

Playing around a bit with the component values in LTSpice delivered a filter with the following parameters:
* Center frequency: 98MHz
* Bandwidth : 20MHz
* L||C = 15nH||150pF
* Coupling capacitor: 33pF

In [3]:
# Double-tuned circuits according to Experimental methods in RF design, Hayward, Campbell, Larkin, p.3.14

F = 98e6 # 98 MHz, center frequency of the FM broadcast band
B = 20e6 # 20 MHz, bandwidth (88-108 MHz)
L = 10/F # H, suggested value for the inductance
Qu = 60 # unloaded Q factor of the inductance
k = 0.707 # coupling coefficient for Butterworth response
q = 1.414 # Q factor for Butterworth response
R0 = 50 # 50 Ohm, characteristic impedance

import math
print(f"L = {L} H")
ω = 2 * math.pi * F
C0 = 1 / (ω**2 * L) # F, intermediate value for the capacitance
C12 = C0 * k * B / F # F, capacitance of the coupling capacitor
print(f"C12 = {C12} F")
QE = q * F * Qu / ( B * Qu - q * F ) # effective Q factor
CE = 1 / (ω * math.sqrt(R0 * QE * ω * L - R0**2)) # F, capacitor between the source/load and the resonator
print(f"CE = {CE} F")
CT = C0 - CE - C12 # F, capacitance of the resonator
print(f"CT = {CT} F")

L = 1.0204081632653061e-07 H
C12 = 3.729387590333861e-12 F
CE = 1.0922294985640286e-11 F
CT = 1.1195558149112024e-11 F


This topology has some disadvantages:
* Requires 7 components
* Limited roll-off on the high side
* Small capacitor values (can be worked around by putting capacitors in parallel, thus using even more components)

The advantage however is that the filter can be tuned by changing the resonator capacitors, as discussed in [Secrets of RF Circuit Design, §MOSFET preselector circuits]()

[Designing Narrow-Bandwidth Ladder Filters by Wes Hayward, w7zoi, April 7, 2019](https://w7zoi.net/filters/genfil.pdf) explains well how to design this filter.

## Conventional, series first band pass filter
* Butterworth, 3rd order
* Chebyshev, 3rd order

### Simulation

<figure>
    <img src='./doc/pre-selector-filter_schematic.png' alt='missing' />
    <figcaption>Schematic of the three filter topologies: OUT1=parallel LC, OUT3=top C-coupled resonator, OUT4=Conventional shunt first BPF</figcaption>
</figure>

<figure>
    <img src='./doc/pre-selector-filter.png' alt='missing' />
    <figcaption>Simulation results of the three filter topologies</figcaption>
</figure>

### Simulation results of the parallel LC bandpass filter
* Center frequency: 95.5MHz
* Insertion loss: 1.4dB
* Return loss : > 27dB
* Lower 3dB frequency: 83.4MHz
* Upper 3dB frequency: 108.9MHz

# Image Filter

In [4]:
def parallel_RLC(Rp, B, f_0):
    ω = 2 * math.pi * f_0
    Q = f_0 / B
    X_0 = Rp / Q
    L = X_0 / ω
    print("\tCalculated L = {:.2e} H".format(L))
    C = 1 / (ω * X_0)
    print("\tCalculated C = {:.2e} F".format(C))
    B = f_0 / Q
    print("\tCalculated B = {:.2e} Hz".format(B))
    return L, C, B

f = 98e6 # 98 MHz, center frequency of the FM broadcast band
B = 20e6 # 20 MHz, bandwidth (88-108 MHz)
print("For Ic = 1mA")
L, C, B = parallel_RLC(3.3e3, B, f)
print("For Ic = 2mA")
L, C, B = parallel_RLC(1.6e3, B, f)
print("For Ic = 3mA")
L, C, B = parallel_RLC(1.1e3, B, f)

For Ic = 1mA
	Calculated L = 1.09e-06 H
	Calculated C = 2.41e-12 F
	Calculated B = 2.00e+07 Hz
For Ic = 2mA
	Calculated L = 5.30e-07 H
	Calculated C = 4.97e-12 F
	Calculated B = 2.00e+07 Hz
For Ic = 3mA
	Calculated L = 3.65e-07 H
	Calculated C = 7.23e-12 F
	Calculated B = 2.00e+07 Hz


Be aware that these are not the final values for the inductors yet.  The impedance matching network for matching to the 50ohm output will add an inductor in parallel, which will reduce the inductance of the collector filter.

# References
* [Episode 1 - Transceivers and Filters - Part 1](https://www.youtube.com/watch?v=r_p7AHsSOdw&list=PL9Ox3wpnB0kqekAyz6blg4YdvoEMoJNJY&index=1&pp=iAQB)
* [Episode 1 - Transceivers and Filters - Part 2](https://www.youtube.com/watch?v=He0-X6FCLMo&list=PL9Ox3wpnB0kqekAyz6blg4YdvoEMoJNJY&index=2&pp=iAQB)
